# 8 queen problem with genetic algorithms


In [ ]:
import numpy as np
import random


In [ ]:
def generate_population(N):
  population = np.arange(0,8)
  totalPop = np.zeros((N,8))
  for i in range(N):
    permutation = np.random.permutation(population)
    totalPop[i] = permutation
  return totalPop


In [ ]:
def get_fitness(population,N):
  fitness = np.zeros((N))
  errorCount = 0
  contI = 0
  for i in population:
    for j in range(len(i)):
        for l in range(len(i)):
          if j != l:
            if checkError(j,l,i):
              errorCount+=1
              break;
    fitness[contI] = (8 - errorCount) + 1
    errorCount = 0
    contI+=1
  return fitness

In [ ]:
def checkError(index1,index2,permutation):
  if ( (index1 + permutation[index1]) == (index2 + permutation[index2]) ) or ( (permutation[index1] - index1) == (permutation[index2] - index2) ):
    return True
  else:
    return False

In [ ]:
def get_elite(fitness,population):
  max = -1
  maxIndex = 0
  for i in range(len(fitness)):
    if fitness[i]>max:
      max = fitness[i]
      maxIndex = i
  return max, population[maxIndex]

In [ ]:
def get_worst(fitness,population):
  max = 10
  maxIndex = 0
  for i in range(len(fitness)):
    if fitness[i]<max:
      max = fitness[i]
      maxIndex = i
  return maxIndex

In [ ]:
def get_parents(fitness, population):
  fitSum = np.sum(fitness)
  proportions = fitness/fitSum
  ranges = np.zeros((len(fitness),2))
  startIndex = 0
  for i in range(len(fitness)):
    ranges[i][0] = startIndex
    ranges[i][1] = startIndex + proportions[i]
    startIndex+=proportions[i]

  newPopulation = np.zeros((len(fitness),8))
  for i in range(len(fitness)):
    randomN = random.random()
    for j in range(len(fitness)):
      if randomN > ranges[j][0] and randomN < ranges[j][1]:
        newPopulation[i] = np.copy(population[j])
  
  return newPopulation

In [ ]:
def get_crossover(population,N,Pr):
  newPopulation = np.zeros((N,8))
  for i in range(N):
    if random.random() < Pr:
      parent1 = population[ np.random.randint(0,N) ]
      parent2 = population[ np.random.randint(0,N) ]
      sections = np.zeros((2))
      sections = np.random.randint(1,7),np.random.randint(1,7)
      sections = np.sort(sections)
      son = np.zeros((8))
      son[:] = -1
      son[sections[0]:sections[1]] = np.copy(parent1[sections[0]:sections[1]])

      for j in range(0,sections[0]):
        if parent2[j] not in son:
          son[j] = np.copy(parent2[j])
      
      for j in range(sections[1],len(parent2)):
        if parent2[j] not in son:
          son[j] = np.copy(parent2[j])

      for j in range(len(parent2)):
        if parent2[j] not in son:
          cont = 0
          while son[cont] !=-1 and cont <8:
            cont+=1
          son[cont] = np.copy(parent2[j])

      newPopulation[i] = np.copy(son)
    else:
      newPopulation[i] = np.copy(population[i])
  
  return newPopulation

In [ ]:
def mutation(population,N,Pm):
  newPopulation = np.zeros((N,8))
  newPopulation = np.copy(population)
  for i in range(N):
    if random.random() < Pm:
      random1, random2 = np.random.randint(0,8),np.random.randint(0,8)
      aux = newPopulation[i][random1]
      newPopulation[i][random1] = np.copy(newPopulation[i][random2])
      newPopulation[i][random2] = aux
  return newPopulation 
  

In [ ]:
def gen_algorithm(N,G,Pr,Pm):
  Gk = 0
  population = generate_population(N)
  fitness = get_fitness(population,N)
  
  elite = get_elite(fitness,population)

  while Gk < G or elite[0] < 9:
    population = np.copy(get_parents(fitness,population))
    population = np.copy(get_crossover(population,N,Pr))
    population = np.copy(mutation(population,N,Pm))
    fitness = get_fitness(population,N)

    newElite = get_elite(fitness,population)

    if newElite[0] > elite[0]:
      elite= newElite[0],newElite[1]
    else:
      worst = get_worst(fitness,population)
      best = np.array(elite[1])
      population[worst] = np.copy(best)
    

    Gk+=1
  return elite
    


In [ ]:
import io

def board_to_fen(board):
    # Use StringIO to build string more efficiently than concatenating
    with io.StringIO() as s:
        for row in board:
            empty = 0
            for cell in row:
                c = cell[0]
                if c in ('w', 'b'):
                    if empty > 0:
                        s.write(str(empty))
                        empty = 0
                    s.write(cell[1].upper() if c == 'w' else cell[1].lower())
                else:
                    empty += 1
            if empty > 0:
                s.write(str(empty))
            s.write('/')
        # Move one position back to overwrite last '/'
        s.seek(s.tell() - 1)
        # If you do not have the additional information choose what to put
        s.write(' w KQkq - 0 1')
        return s.getvalue()

In [ ]:
final_solution = gen_algorithm(30,100,0.8,0.3)
print("FINAL SOLUTION: ",final_solution)

tablero = np.zeros((8,8))
print()
for i in range(len(final_solution[1])):
  index = int(final_solution[1][i])
  tablero[i][index] = '1'

print("---SOLVED CHESSBOARD---")
print(tablero)

board = [
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
    ['em', 'em', 'em', 'em', 'em', 'em', 'em', 'em'],
]
for i in range(8):
  for j in range(8):
    if tablero[i][j] == 1:
      board[i][j] = 'bq'

print("PUEDES COPIAR ESTE STRING Y PEGARLO EN CHESS.COM, EN LA PARTE DONDE DICE 'LOAD FEN'\nPARA UNA MEJOR REPRESENTACIÓN GRÁFICA")
print(board_to_fen(board))

FINAL SOLUTION:  (9.0, array([1., 3., 5., 7., 2., 0., 6., 4.]))

---SOLVED CHESSBOARD---
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]
PUEDES COPIAR ESTE STRING Y PEGARLO EN CHESS.COM, EN LA PARTE DONDE DICE 'LOAD FEN'
PARA UNA MEJOR REPRESENTACIÓN GRÁFICA
1q6/3q4/5q2/7q/2q5/q7/6q1/4q3 w KQkq - 0 1
